In [3]:
import configobj
from crate import client
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt

import pymc3 as pm
import theano.tensor as tt

from teams import nba_teams
team_keys = nba_teams

def index_teams(team_keys):
    team_index = {}
    for i,key in enumerate(team_keys):
        team_index[team_keys[key]] = i
        
    return team_index
    
team_index = index_teams(team_keys)

print team_index

{'MIL': 18, 'GSW': 10, 'MIN': 14, 'MIA': 7, 'WSH': 12, 'ATL': 26, 'BOS': 6, 'DET': 5, 'NYK': 11, 'DEN': 29, 'DAL': 13, 'BKN': 27, 'POR': 9, 'OKC': 16, 'TOR': 20, 'CLE': 25, 'SAS': 3, 'CHA': 17, 'UTA': 21, 'CHI': 1, 'HOU': 4, 'LAL': 0, 'PHI': 2, 'MEM': 19, 'LAC': 15, 'SAC': 23, 'ORL': 8, 'PHX': 22, 'IND': 28, 'NOP': 24}


In [ ]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

sql = '''
SELECT 
nba.games."GameId" as "GameId", 
nba.games."GameDate" as "GameDate", 
nba.games."HomeTeam" as "HomeTeam", 
nba.games."AwayTeam" as "AwayTeam",
nba.results."AwayScore" as "AwayScore", 
nba.results."HomeScore" as "HomeScore",
nba.lines."HomeSpread" as "HomeSpread", 
nba.lines."OverUnder" as "OverUnder",
nba.predictions."HomeScore" as "PredictedHomeScore",
nba.predictions."AwayScore" as "PredictedAwayScore"
FROM nba.games, nba.lines, nba.results, nba.predictions
WHERE nba.games."GameId" = nba.results."GameId"
AND nba.games."GameId" = nba.lines."GameId"
AND nba.games."GameId" = nba.predictions."GameId"
AND nba.games."GameDate" > 20151025
ORDER BY nba.games."GameDate" DESC
'''

df = pd.read_sql(sql, connection)

df_final = df.drop_duplicates(["GameId"], take_last=True)
#print df_final

results_final = df_final.to_dict('records')

results = []

# calculate if we won or lost
for game in results2:
    
    # check if we predicted the game correctly
    home_minus_away = int(game['HomeScore'] - game['AwayScore'])
    home_minus_away_predicted = int(game['PredictedHomeScore'] - game['PredictedAwayScore'])
    if ((home_minus_away ^ home_minus_away_predicted) >= 0): #check if the numbers have the same sign
        game['GameOutcome'] = "Win"
    else:
        game['GameOutcome'] = "Lose"
        
    # check if we predicted the over/under correctly   
    if (((game['HomeScore'] + game['AwayScore']) > game['OverUnder']) and \
    ((game['PredictedHomeScore'] + game['PredictedAwayScore']) > game['OverUnder'])) or \
    (((game['HomeScore'] + game['AwayScore']) < game['OverUnder']) and \
    ((game['PredictedHomeScore'] + game['PredictedAwayScore']) < game['OverUnder'])):
        game['OverUnderOutcome'] = "Win"
    else:
        game['OverUnderOutcome'] = "Lose"
        
    # check if we predicted the spread correctly
    if (((game['AwayScore'] - game['HomeScore']) < game['HomeSpread']) and \
    ((game['PredictedAwayScore'] - game['PredictedHomeScore']) < game['HomeSpread'])) or \
    (((game['AwayScore'] - game['HomeScore']) > game['HomeSpread']) and \
    ((game['PredictedAwayScore'] - game['PredictedHomeScore']) > game['HomeSpread'])):
        game['SpreadOutcome'] = "Win"
    else:
        game['SpreadOutcome'] = "Lose"   
        
    results.append(game)

In [ ]:
bins = np.linspace(-30,30,60)
a = [(game['PredictedAwayScore'] - game['PredictedHomeScore']) - game["HomeSpread"] for game in results if game['SpreadOutcome']=='Win']
b = [(game['PredictedAwayScore'] - game['PredictedHomeScore']) - game["HomeSpread"] for game in results if game['SpreadOutcome']=="Lose"]
plt.xlabel("Predicted - Offered Point Spread")
plt.hist(a,bins,histtype='step',color='g')
plt.hist(b,bins,histtype='step',color='r')
plt.show()

bins = np.linspace(-30,30,60)
a = [(game['PredictedHomeScore'] + game['PredictedAwayScore']) - game['OverUnder'] for game in results if game['OverUnderOutcome']=='Win']
a = [(game['PredictedHomeScore'] + game['PredictedAwayScore']) - game['OverUnder'] for game in results if game['OverUnderOutcome']=='Lose']


plt.hist(a,bins,histtype='step')
plt.xlabel("Predicted - Offered O/U")
plt.show()

bins = np.linspace(-50,50,50)
a = [(game['HomeScore'] + game['AwayScore']) - game['OverUnder'] for game in results]
b = [(game['HomeScore'] + game['AwayScore']) - (game['PredictedHomeScore'] + game['PredictedAwayScore']) for game in results]
plt.hist(a,bins,histtype='step')
plt.hist(b,bins,histtype='step')
plt.xlabel("Actual -  Predicted O/U")
plt.show()

bins = np.linspace(-50,50,50)
a = [game['HomeSpread'] - (game['HomeScore'] - game['AwayScore']) for game in results]
b = [(game['PredictedHomeScore'] - game['PredictedAwayScore']) - (game['HomeScore'] - game['AwayScore']) for game in results]
plt.hist(a,bins,histtype='step',color='b')
plt.hist(b,bins,histtype='step',color='g')
plt.xlabel("Actual -  Predicted Spread")
plt.show()

In [5]:
config = configobj.ConfigObj("/Users/smacmullin/sports/crate.ini")
crate_host = config["crate"]["host_url"]

connection = client.connect(crate_host)
print connection.client._active_servers
cursor = connection.cursor()

def generate_test_times():

    start = dt.date(2015, 10, 01)

    test_times = [int(start.strftime('%Y') + start.strftime('%m') + start.strftime('%d'))]

    delta = dt.timedelta(days=1)

    for d in range(250):

        t = start + delta

        test_times.append(int(t.strftime('%Y') + t.strftime('%m') + t.strftime('%d')))

        start = t

    return test_times

test_times = generate_test_times()

# #look for back to back games, find avg difference in night 2 - night 1 pts scored vs points allowed

for team in team_index.keys():
    
    for date in test_times:
    
        sql = '''
        SELECT
        nba.games."GameId" as "GameId",
        nba.games."GameDate" as "GameDate",
        nba.games."HomeTeam" as "HomeTeam",
        nba.games."AwayTeam" as "AwayTeam",
        nba.results."AwayScore" as "AwayScore",
        nba.results."HomeScore" as "HomeScore",
        FROM nba.games, nba.results
        WHERE nba.games."GameId" = nba.results."GameId"
        AND (nba.games."HomeTeam" = '%s' or nba.games."AwayTeam" = '%s')
        AND nbg.games."GameDate"=%s
        ''' % (team, team, date)

        df = pd.read_sql(sql, connection)

['http://localhost:4200']
[20151001, 20151002, 20151003, 20151004, 20151005, 20151006, 20151007, 20151008, 20151009, 20151010, 20151011, 20151012, 20151013, 20151014, 20151015, 20151016, 20151017, 20151018, 20151019, 20151020, 20151021, 20151022, 20151023, 20151024, 20151025, 20151026, 20151027, 20151028, 20151029, 20151030, 20151031, 20151101, 20151102, 20151103, 20151104, 20151105, 20151106, 20151107, 20151108, 20151109, 20151110, 20151111, 20151112, 20151113, 20151114, 20151115, 20151116, 20151117, 20151118, 20151119, 20151120, 20151121, 20151122, 20151123, 20151124, 20151125, 20151126, 20151127, 20151128, 20151129, 20151130, 20151201, 20151202, 20151203, 20151204, 20151205, 20151206, 20151207, 20151208, 20151209, 20151210, 20151211, 20151212, 20151213, 20151214, 20151215, 20151216, 20151217, 20151218, 20151219, 20151220, 20151221, 20151222, 20151223, 20151224, 20151225, 20151226, 20151227, 20151228, 20151229, 20151230, 20151231, 20160101, 20160102, 20160103, 20160104, 20160105, 201

In [9]:
for i in range(len(test_times)):
    print test_times.next()

AttributeError: 'list' object has no attribute 'next'